In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(rf'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [2]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [3]:
medium = td.Medium(permittivity=45)
run = True
a=1.5873
lambdas =  a/np.array([0.12,0.5])
3e10/(lambdas)  * 1e-9 #frequencies of experiment 

array([2.26800227, 9.45000945])

In [4]:
folder_path=rf"H:\phd stuff\tidy3d\structures\2D SHU"
air_gap = 0.015 #in cm
sample = 1
project_name = fr"20250729 {air_gap*10:.2f}mm Gap Cavity 3D"
empty=False
runtime_ps = 10e-12
min_steps_per_lambda = 28
id0 = ""
ref_slices = []
slab_size_x = 55

In [5]:
count = 0
for l,file in enumerate(natsorted(os.listdir(folder_path))):
    print(Path(file).stem)
    if Path(file).stem not in ["chi_0.37_N_10000_posics"]:
        continue
    data = []
    centers = []

    with open(os.path.join(folder_path, file),"r") as f: 
        lines = f.read().splitlines() 
        i=0
        index = 1
        num_samples = int(lines[index].split()[0])

        while num_samples>0 and i < sample+1:
            try:
                num_samples = int(lines[index].split()[0])
                info = lines[index]
                old_index = index
                index += num_samples+1
                li = (lines[old_index+1:index])
                centers = np.array([list(map(float, item.split())) for item in li])
                data += [{
                    "centers":centers,
                    "Lx":float(lines[old_index].split()[1]),
                    "Ly":float(lines[old_index].split()[2]),
                    "points":float(lines[old_index].split()[0]),
                }]
                i+=1
            except:
                num_samples = 0
    

    for k,item in enumerate(data):
        if k >= sample:
            continue
        

        structure_1 = AM.loadAndRunStructure(key = tidy3dAPI
                    ,direction="z", lambda_range=lambdas,
                    box_size= data[k]['Lx'],runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                   scaling=1,shuoff_condtion=1e-20, verbose=True,
                   monitors=["flux"],
                   freqs=700, 
                   source="planewave", 
                   width=0.4, ref_only=True
                   )


        sim = structure_1.sim



        boundaries= td.BoundarySpec(
                x=td.Boundary(plus=td.Absorber(num_layers=160),minus=td.Absorber(num_layers=160)),
                y=td.Boundary(plus=td.Absorber(num_layers=160),minus=td.Absorber(num_layers=160)),
                z=td.Boundary(plus=td.PECBoundary(),minus=td.PECBoundary()),
                # z=td.Boundary.periodic(),
            )
        sim = sim.copy(update={'boundary_spec':boundaries})
        sim = sim.copy(update={'sources':[]})
        sim = sim.copy(update={'monitors':[]})


        slices= [0.07]
        for l,slicing in enumerate(slices):
         x_positions =  np.arange(-5.5, 6.5, 1)
         for n,x_position in enumerate(x_positions):
            run_name = f"{Path(file).stem}_emmiter_{x_position:.4f} - Sample_{k}"
            print(run_name)
            count = count+1
            Lx,Ly,Lz=110,70,0.5+air_gap
            sim = sim.copy(update={'size':[Lx,Ly,Lz]})


            sources = []
            y_position = -100/2*a*slicing-22.5
            
            sources.append(
                    td.PointDipole(
                        center=(x_position, y_position,Lz/2 - 0.2), 
                        source_time=td.GaussianPulse(freq0=structure_1.freq0, fwidth=structure_1.freqw),
                        polarization='Ez'
                    )
                )
            ###Antenna structure########
            antenna = td.Structure(geometry=td.Cylinder(center=[x_position, y_position, Lz/2 - 0.1], radius=0.03, length=0.2),medium=td.PECMedium())
            ####################


            sim = sim.copy(update={'sources':sources})
            

            

            time_monitorFieldOut = td.FieldTimeMonitor(
                        center = (0,0,0),
                        size = (Lx,Ly,0),
                            start=0,
                            stop=structure_1.t_stop,
                            interval=50,
                            fields=["Ex", "Ey", "Ez"],
                            name="time_monitorFieldOut",

                        )


            freq_monitorFieldOut = td.FieldMonitor(
                        center = (0,0,Lz/2 - 0.2),
                        size = (Lx, Ly,0),
                           freqs =structure_1.monitor_freqs,
                            fields=["Ex", "Ey", "Ez"],
                            name="freq_monitorFieldOut",

                        )

           
            monitor_1 = td.FluxMonitor(
                    center = (
                                    0,-((Lx*slicing)/2+3),0
                                    ),
                    size = (
                       td.inf,0,td.inf
                        ),
                    freqs = structure_1.monitor_freqs,
                    name='flux2' )

            monitor_2 = td.FluxMonitor(
                    center = (
                                    0,(Lx*slicing)/2+3,0
                                    ),
                    size = (
                       td.inf,0,td.inf
                        ),
                    freqs = structure_1.monitor_freqs,
                    name='flux1' 
                )

            sim = sim.copy(update={'monitors':[freq_monitorFieldOut,monitor_1,monitor_2]})


            cyl_group = []
            for x,y in data[k]['centers']:
                x,y = x*a,y*a
                if np.abs(y)<=(100/2*a*slicing) and np.abs(x)<=(slab_size_x/2 * a):
                    cyl_group.append(td.Cylinder(center=[x, y, (-Lz+0.5)/2 ], radius=0.189*a, length=0.5))

            cylinders = td.Structure(geometry=td.GeometryGroup(geometries=cyl_group), medium=medium)

            
            if empty:
                sim = sim.copy(update={'structures':[],"grid_spec": td.GridSpec.uniform(dl=structure_1.dl)})
            else:
                sim = sim.copy(update={'structures':[cylinders,antenna],"grid_spec": td.GridSpec.uniform(dl=structure_1.dl)})


            # fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
            # sim.plot(z=0.15, ax=ax)
            # plt.show()
            # sim.plot_3d()
            sim_name = run_name
            if run:
                folder_run = Path(file).stem + fr"_L={100*a*slicing:.2g}"
                file_path = rf"H:\phd stuff\tidy3d\data/{project_name}/{Path(file).stem}/{folder_run}/{sim_name}.txt"
                
                if os.path.exists(file_path):
                    print("Exist!!")
                else:
                    try:
                        id0 = ref_slices[n]
                    except:
                        id0 =web.upload(sim.copy(update={'structures':[]}), folder_name=project_name,task_name=fr"{sim_name}_ref", verbose=True)
                        web.start(task_id = id0)
                        ref_slices += [id0]


                    id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
                    ids = id0+ '\n' + id
                    # Check if the folder exists
                    if not os.path.exists( fr"H:\phd stuff\tidy3d\data/{project_name}/{Path(file).stem}/{folder_run}/"):
                        os.makedirs(fr"H:\phd stuff\tidy3d\data/{project_name}/{Path(file).stem}/{folder_run}/")
                        print(fr"Folder 'H:\phd stuff\tidy3d\data/{project_name}/{Path(file).stem}/{folder_run}/' created successfully.")

                    # Open file in write mode
                    with open(file_path, "w") as f:
                        # Write the string to the file
                        f.write(ids)

                    web.start(task_id = id)
                    
                    # raise TypeError("Program ends here")
                
            else:
                # id =web.upload(sim,task_name=sim_name, verbose=True)
                # print(web.estimate_cost(id))
                raise TypeError("Program ends here")



chi_0.20_N_10000_posics
chi_0.30_N_10000_posics
chi_0.31_N_10000_posics
chi_0.32_N_10000_posics
chi_0.33_N_10000_posics
chi_0.34_N_10000_posics
chi_0.35_N_10000_posics
chi_0.36_N_10000_posics
chi_0.37_N_10000_posics
Configured successfully.
chi_0.37_N_10000_posics_emmiter_-5.5000 - Sample_0


14:06:30 W. Europe Daylight Time WARNING: Structure at 'structures[0]' has      
                                 bounds that extend exactly to simulation edges.
                                 This can cause unexpected behavior. If         
                                 intending to extend the structure to infinity  
                                 along one dimension, use td.inf as a size      
                                 variable instead to make this explicit.        

                                 WARNING: Suppressed 1 WARNING message.         

                                 WARNING: Monitor 'freq_monitorFieldOut'        
                                 estimated storage is 10.16GB. Consider making  
                                 it smaller, using fewer frequencies, or spatial
                                 or temporal downsampling using 'interval_space'
                                 and 'interval', respectively.                  

14:06:31 W. Europe Daylight Time Created task                                   
                                 'chi_0.37_N_10000_posics_emmiter_-5.5000 -     
                                 Sample_0_ref' with task_id                     
                                 'fdve-5fe29e5f-4a00-4a6a-a9b5-e7fb97de5cde' and
                                 task_type 'FDTD'.

...

                                 View task using web UI at                      
                                 ]8;id=325708;https://tidy3d.simulation.cloud/workbench?taskId=fdve-87dcaf77-9485-4dbb-adfb-d306e18f59ed\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=413926;https://tidy3d.simulation.cloud/workbench?taskId=fdve-87dcaf77-9485-4dbb-adfb-d306e18f59ed\task]8;;\
                                 ]8;id=413926;https://tidy3d.simulation.cloud/workbench?taskId=fdve-87dcaf77-9485-4dbb-adfb-d306e18f59ed\Id]8;;\]8;id=325708;https://tidy3d.simulation.cloud/workbench?taskId=fdve-87dcaf77-9485-4dbb-adfb-d306e18f59ed\=]8;;\]8;id=694252;https://tidy3d.simulation.cloud/workbench?taskId=fdve-87dcaf77-9485-4dbb-adfb-d306e18f59ed\fdve]8;;\]8;id=325708;https://tidy3d.simulation.cloud/workbench?taskId=fdve-87dcaf77-9485-4dbb-adfb-d306e18f59ed\-87dcaf77-9485-4dbb-adfb-d306e18f59ed']8;;\.

                                 Task folder: ]8;id=224339;https://tidy3d.simulation.cloud/folders/folder-13cea8cf-8d12-46c4-be41-e4c132777873\'20250729 0.15mm Gap Cavity 3D']8;;\.

Output()

14:08:06 W. Europe Daylight Time Maximum FlexCredit cost: 0.803. Minimum cost   
                                 depends on task execution details. Use         
                                 'web.real_cost(task_id)' to get the billed     
                                 FlexCredit cost after a simulation run.

chi_0.38_N_10000_posics
chi_0.39_N_10000_posics
chi_0.40_N_10000_posics
